In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
from shapely.geometry import Polygon, Point, LineString
import pandas as pd
import geopandas as gpd
import folium
from IPython.display import Image
import selenium.webdriver
import shapely.geometry
import osmnx as ox
import pyproj
from geojson.feature import Feature, FeatureCollection
from shapely.geometry import Point, Polygon, LineString
import h3
import json
import branca.colormap as cmp
import glob
from statistics import *

In [3]:
import os
cwd = os.getcwd()
os.chdir(os.path.join(cwd,'..\..\..'))
import ExMAS.main
import ExMAS.utils

from ExMAS.utils import inData as inData

In [132]:
def ride_finder(dem, disc):
    dem = "_"+str(dem)+"_"
    disc= "_"+str(disc).replace(".","")+"_"
    rides = glob.glob('ExMAS/data/results3/rides/*.csv', recursive=True)
    rides  = [s for s in rides if dem in s]
    rides = [s for s in rides if disc in s]
    return rides

def req_finder(dem, disc):
    dem = "_"+str(dem)+"_"
    disc= "_"+str(disc).replace(".","")+"_"
    req = glob.glob('ExMAS/data/results3/requests/*.csv', recursive=True)
    req  = [s for s in req if dem in s]
    req = [s for s in req if disc in s]
    return req
    

In [121]:
def no_repli_format(df_ride, df_req, index = 0): 
    #function to use in case you want to make a map without considering replications
    
    resop = pd.DataFrame(columns=['lambda_r','origin','destination'])
    #resop = resop.fillna(0)


    file_ride = pd.read_csv(df_ride[index])                         #opening the rides csv file
    file_req = pd.read_csv(df_req[index])                           #opening the requests csv file
    org  = file_req['origin']                                       #loading the origins of requests
    dest = file_req['destination']                                  #lodading destinations of requests
    for i in range(len(file_ride)):                                 #looping thhrough the indeces in the file
        a = file_ride['lambda_r'].iloc[i]                           #assigning 
        b = json.loads(file_ride['indexes_orig'].iloc[i])           
        c = json.loads(file_ride['indexes_dest'].iloc[i])           #loading indeces of origins and destinations to match with req file
        deg = file_ride['kind'].iloc[i]                             #assigning ride kind 

        if 20<=deg<30:
            deg=2
        elif 30<=deg<40:
            deg=3
        elif 40<=deg<50:
            deg=4
        elif deg>=50:
            deg=5
        else:
            deg=1
        for j in range(len(b)):                                    #for loop loops through origins of travellers being picked up in shared
            resop = resop.append({'lambda_r':a,'origin':org[b[j]],'destination':dest[c[j]],'x':org[b[j]],'y':org[b[j]],'degree':deg},ignore_index=True)

#         resop = resop.append({'lambda_r':a,'origin':org[b[0]],'destination':dest[c[0]],\  #adding only the first passenger picked up
#                                   'x':org[b[0]],'y':org[b[0]],'degree':deg},ignore_index=True)             
            
    resop['x'] = resop['x'].apply(lambda row: inData.nodes.loc[row].x)
    resop['y'] = resop['y'].apply(lambda row: inData.nodes.loc[row].y)


    #resop = pd.concat(resop)
    resop_s = resop[resop['degree']>1]  #filter only sharing rides
    return resop

In [151]:
def repli_format(df_ride, df_req):
    #function to use in case you do not want to make a map without considering replications
    resop = pd.DataFrame(columns=['lambda_r','origin','destination'])
    #resop = resop.fillna(0)

    for req,file in enumerate(df_ride):                               #looping through all the replications

        file_ride = pd.read_csv(file)                                 #reading a specific file
        file_req = pd.read_csv(df_req[req])
        org  = file_req['origin']
        dest = file_req['destination']
        for i in range(len(file_ride)):                               #looping through all the rows
            a = file_ride['lambda_r'].iloc[i]                         
            b = json.loads(file_ride['indexes_orig'].iloc[i])
            c = json.loads(file_ride['indexes_dest'].iloc[i])
            deg = file_ride['kind'].iloc[i]

            if 20<=deg<30:
                deg=2
            elif 30<=deg<40:
                deg=3
            elif 40<=deg<50:
                deg=4
            elif deg>=50:
                deg=5
            else:
                deg=1
            for j in range(len(b)):                                                              #loop adds all the passenger origins to dataframe
                resop = resop.append({'lambda_r':a,'origin':org[b[j]],'destination':dest[c[j]],\
                                      'x':org[b[j]],'y':org[b[j]],'degree':deg},ignore_index=True)
            
#             resop = resop.append({'lambda_r':a,'origin':org[b[0]],'destination':dest[c[0]],\  #adding only the first passenger picked up
#                                   'x':org[b[0]],'y':org[b[0]],'degree':deg},ignore_index=True) 
            
    resop['x'] = resop['x'].apply(lambda row: inData.nodes.loc[row].x)                           #applying x/y coordinates
    resop['y'] = resop['y'].apply(lambda row: inData.nodes.loc[row].y)


    #resop = pd.concat(resop)
    resop_s = resop[resop['degree']>1]  #filter only sharing rides
    return resop_s

In [149]:
def plot_hex_map(inData,res,dem,disc, APERTURE_SIZE = 8, threshold = 1, col = 'u_diff'):
    res['hex_o_{}'.format(APERTURE_SIZE)] = res.apply(lambda row: h3.geo_to_h3(row.y,row.x,APERTURE_SIZE),axis = 1) 
    trips = res
    
    col_geom = 'hex_o_{}'.format(APERTURE_SIZE)
    hexes = pd.Series(list(set(list(trips[col_geom].unique())+list(trips[col_geom].unique())))).to_frame(col_geom)
    hexes = hexes.set_index(col_geom)
    hexes[col_geom] = hexes.index.copy()
    hexes['nobs'] = trips.groupby(col_geom).size()
    hexes = hexes[hexes['nobs']>threshold]
    hexes['geom'] = hexes.apply(lambda x: {"type": "Polygon","coordinates": [h3.h3_to_geo_boundary(h = x[col_geom], geo_json = True)]}, axis = 1)
    aggr = trips.groupby(col_geom)[col]
    hexes['nobs'] = aggr.size()
    hexes[col] = aggr.mean()
    hexes[col+'_std'] = aggr.std()
    list_features = []
    for i, row in hexes.iterrows():
        feature = Feature(geometry = row["geom"],
                          id = row[col_geom],
                          properties = {"resolution": 9})
        list_features.append(feature)

    feat_collection = FeatureCollection(list_features)
    geojson_hexes = json.dumps(feat_collection)
    CENTER = list(inData.nodes.loc[inData.stats.center][['y','x']].values)
    tile = 'cartodbpositron'
    linear = cmp.LinearColormap(['red', 'yellow', 'green'],vmin=min(res[col]), 
                                vmax=max(res[col]),caption='Color Scale for Map') #Caption for Color scale or Legend
    base_map = folium.Map(location=CENTER, zoom_start=12,tiles=tile, zoomControl =  False)
    bins = [min(hexes[col]),0,0.25*max(hexes[col]),0.5*max(hexes[col]),0.75*max(hexes[col]),max(hexes[col])]
            
    m = folium.Choropleth(geo_data = geojson_hexes,data = hexes, columns = [col_geom,col],key_on ="feature.id",
            fill_color='YlGn', control = False,bins=bins,
            fill_opacity=0.7, line_opacity=0.1,legend_name=col+" for demand {} at discount {}".format(dem,disc)).add_to(base_map)
    '''m = folium.Choropleth(geo_data = geojson_hexes,data = hexes, columns = [col_geom,col],key_on ="feature.id",
            fill_color=linear, control = False,
            fill_opacity=0.7, line_opacity=0.1,legend_name=col).add_to(base_map)'''
    m.save('map.html')
    return base_map
    

In [7]:
params = ExMAS.utils.get_config('ExMAS/data/configs/my_config.json') # load the default 

In [8]:
inData = ExMAS.utils.load_G(inData, params, stats=True)  # download the graph

In [9]:
inData = ExMAS.utils.load_albatross_csv(inData, params, sample=True)

In [128]:
demand = 600
discount   = 0.15


In [133]:
df_rides = ride_finder(demand,discount)
df_reqs  = req_finder(demand,discount)

In [147]:
resop_s = repli_format(df_rides,df_reqs)

In [291]:
linear = cmp.LinearColormap(['red', 'yellow', 'green'],vmin=min(resop['lambda_r']), vmax=max(resop['lambda_r']),caption='Color Scale for Map')
#style_function=lambda feature: 
 #                           'fillColor': linear(resop[feature['lambda_r']]),
colour = resop['lambda_r'].apply(lambda x: linear(x))

# Discount = 0.15

First map when the location lambda_r of the first passenger is only accounted for (ORIGIN)

In [145]:
plot_hex_map(inData, resop_s,demand,discount, APERTURE_SIZE = 8, col = 'lambda_r')

Map when the location lambda_r of ALL passengers is accounted for (ORIGIN)

In [148]:
plot_hex_map(inData, resop_s,demand,discount, APERTURE_SIZE = 8, col = 'lambda_r')

# Discount = 0.2

# Discount = 0.25

In [297]:
discount   = 0.25
ride_finder(demand,discount)

'ExMAS/data/results/rides\\city_Amsterdam_Netherlands_nP_200_shareddiscount_025_horizon_1200_dt_0_days_000149027790_searchspace_3542-ride.csv'

# Discount = 0.3

In [301]:
discount   = 0.3
ride_finder(demand,discount)

'ExMAS/data/results/rides\\city_Amsterdam_Netherlands_nP_200_shareddiscount_03_horizon_1200_dt_0_days_000126750103_searchspace_7182-ride.csv'

# Discount = 0.35

In [305]:
discount   = 0.35
ride_finder(demand,discount)

'ExMAS/data/results/rides\\city_Amsterdam_Netherlands_nP_200_shareddiscount_035_horizon_1200_dt_0_days_000305744054_searchspace_12247-ride.csv'